In [24]:
from IPython.display import HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# Uploading to SynBioHub

Using this notebook will allow you to visualize your Excel data and store it in SynBioHub. First, make sure your Excel spreadsheet follows the template provided by the TASBE-Flow Analytics Tutorial: https://github.com/TASBE/TASBEFlowAnalytics-Tutorial/blob/master/batch_template.xlsm

After downloading the accompanying python code to this notebook, make sure that it is in the same directory as your Excel file. See the README for more detailed instructions.

In [25]:
from sbol import *
import re
import sys
import xlrd
from ipywidgets import widgets
import getpass

## Select File:

Select the Excel spreadsheet you want to model.

In [26]:
from IPython.display import display, clear_output
from ipyupload import FileUpload

w = FileUpload(
    accept='',
    multiple=False,
    disabled=False,
    style_button='color: white; background-color: darkorange; width: 180px;', 
    compress_level=0
)

display(w)

FileUpload(style_button='color: white; background-color: darkorange; width: 180px;')

## Checking File Compatability:
Press the button below to confirm upload:

In [27]:
from tqdm import tqdm
import time
import os

ActionButton = widgets.Button(
    description = 'Proceed'
)
out = widgets.Output()
def on_button_clicked(_):
    with out:
        clear_output()
    if w.value == {}:
        print("You must upload a file first.")
        return
    else:
        with tqdm(total=1000) as pbar:
            file_location = list(w.value)
            file_location = file_location[0]
            pbar.set_description("Converting \"%s\" to SBOL..." % file_location)
            pbar.update(48)
            time.sleep(0.1)
            name,extension = os.path.splitext(file_location)
            counter = 0
            for i in ['.xlsm','.xlsx','.xltx','.xltm','.xls','.xlm','.xlt']:
                if extension == i:
                    counter = 1
                    break
            if counter != 1:
                print('Error: "%s" file format is not supported.' % extension)
            else:    
                %run -i 'SBOLconverter.py'
                pbar.update(48)
                time.sleep(0.1)

                global doc
                pbar.update(48)
                time.sleep(0.1)         
                doc = Document()
                pbar.update(48)
                time.sleep(0.1)
                setHomespace('http://bu.edu/dasha')       
                pbar.update(48)
                time.sleep(0.1)    
                Config.setOption('sbol_typed_uris',False)
                pbar.update(48)
                time.sleep(0.1)
                Config.setOption('sbol_compliant_uris',True)
                pbar.update(48)
                time.sleep(0.1)

                wb = MakeBook(file_location)
                pbar.update(48)
                time.sleep(0.1)
                ExperimentSheetName = 'Experiment DNA sample' #future: allow user to enter sheet name, store it here
                ExpSheet = ExpSheetFinder(wb,ExperimentSheetName)
                if ExpSheet == -1:
                    return
                pbar.update(40)
                time.sleep(0.1)
                ExpName = ExpNameFinder(wb)
                if ExpName == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                Unit = UnitCollectionFunc(ExpSheet)
                if Unit == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                (ModList,PlasmidList_orig) = PlasModList(ExpSheet)
                if (ModList == -1 and PlasmidList_orig == -1):
                    return
                pbar.update(48)
                time.sleep(0.1)
                PlasmidList_norepeat = PlasNoRepeat(PlasmidList_orig)
                pbar.update(48)
                time.sleep(0.1)
                newModList = ModListCleaner(ModList,ExpName)
                pbar.update(48)
                time.sleep(0.1)
                ModDefDict = ModMaker(ModList,newModList,ExpSheet,doc)
                if ModDefDict == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                SampleSheet = SamplesSheetFinder(wb)
                if SampleSheet == -1:
                    return
                (SampleList, SampleDescriptions) = SampleListDesc(SampleSheet)
                if SampleList == -1 and SampleDescriptions == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                ConditionDictionary = SampleExpConditions(SampleSheet, SampleList)
                if ConditionDictionary == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                (SampleModDefDict, newSampleList) = SampleModMaker(SampleSheet, SampleList, SampleDescriptions, ConditionDictionary, ExpName,doc)
                if SampleModDefDict == -1 and newSampleList == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                ret = ModAdder(SampleList, newSampleList, SampleModDefDict, ModList, newModList, ModDefDict, ConditionDictionary)
                if ret == -1:
                    return
                pbar.update(48)
                time.sleep(0.1)
                CompDefDict = CompMaker(PlasmidList_norepeat,doc)
                pbar.update(48)
                time.sleep(0.1)
                ret = FuncMaker(ModList, newModList, ModDefDict, CompDefDict, ExpSheet, Unit)
                pbar.update(48)
                time.sleep(0.1)

                print('Success!')

ActionButton.on_click(on_button_clicked) 
ActionButton


Button(description='Proceed', style=ButtonStyle())

Converting "20180731 JHT14.xlsm" to SBOL...: 100%|██████████| 1000/1000 [00:12<00:00, 21.60it/s]

Success!


## Enter Project Details:
Enter the **displayID** (REQUIRED, alphanumeric and underscore characters only, cannot begin with a digit), **project name** (optional, if you want to elaborate on the displayID), **project description**, and **version** (REQUIRED for the collection URI) of the Project you want to upload to. If it already exists, entering only the displayID and version is sufficient.

In [28]:
displaywidget = widgets.Text(
    value='',
    description = 'displayID'
)
namewidget = widgets.Text(
    value='',
    description = 'name'
)
descriptionwidget = widgets.Textarea(
    value='',
    description = 'description'
)
versionwidget = widgets.Text(
    description = 'version',
    placeholder = 'for example 1.0.0 or 1'
)
collectionbox = widgets.VBox([displaywidget,namewidget,descriptionwidget,versionwidget])
collectionbox


## Enter Experiment Details:
Enter the **displayID** (REQUIRED, alphanumeric and underscore characters only, cannot begin with a digit), **experiment name** (optional, if you want to elaborate on the displayID), and the **experiment description** of the new Experiment you will be uploading.

In [29]:
subIDwidget = widgets.Text(
    value='',
    description = 'displayID'
)
subcolnamewidget = widgets.Text(
    value='',
    description = 'name'
)
subdescriptionwidget = widgets.Textarea(
    value='',
    description = 'description'
)

subcollectionbox = widgets.VBox([subIDwidget,subcolnamewidget,subdescriptionwidget])
subcollectionbox


## Enter Login Information:
Enter your **username** and **password** for the SynBioHub account you want to upload your collection to.

In [30]:
usernamewidget = widgets.Text(
    value='',
    placeholder='',
    description='Username:',
    disabled=False
)
# loginButton = widgets.Button(
#     description = 'Login'
# )
#^if in the future login comes before actual upload
uploadButton = widgets.Button(
    description = 'Upload to SynBioHub'
)

passwordwidget = widgets.Password(
    description = 'Password:'
)

yesButton = widgets.Button(
    description = 'Yes'
)

noButton = widgets.Button(
    description = 'No'
)

YesNoBox = widgets.HBox([yesButton, noButton])

def YesButtonClicked(_):
    formatlist = [displaywidget.value,subIDwidget.value]
    print('Creating a new project with displayID "{}" containing an experiment with displayID "{}".'.format(*formatlist))
    doc.displayId = displaywidget.value
    doc.name = namewidget.value
    doc.description = descriptionwidget.value
    doc.version = versionwidget.value
    NewProjUpload(usernamewidget.value,passwordwidget.value,doc)
    sep = '@'
    text = usernamewidget.value 
    rest = text.split(sep, 1)[0]
    rootcolURI = "https://synbiohub.org/user/" + rest + "/" + displaywidget.value + "/" + displaywidget.value + "_collection/" + versionwidget.value
    print(rootcolURI)
    return
    
def NoButtonClicked(_):
    print('Upload stopped.')
    return

out = widgets.Output()

def on_button_clicked(_):
    sep = '@'
    text = usernamewidget.value 
    rest = text.split(sep, 1)[0]
    rootcolURI = "https://synbiohub.org/user/" + rest + "/" + displaywidget.value + "/" + displaywidget.value + "_collection/" + versionwidget.value
    ret = UploadFunc(usernamewidget.value,passwordwidget.value,subIDwidget.value,subcolnamewidget.value,subdescriptionwidget.value,rootcolURI,doc)
    if ret == 1:
        print('No project with the displayID "{}" found.'.format(displaywidget.value))
        print('Do you want to create a new project with this displayID?')
        yesButton.on_click(YesButtonClicked)
        noButton.on_click(NoButtonClicked)
        display(YesNoBox)
    elif ret == 2:
        print(rootcolURI)
        return
    else:
        return
    
uploadButton.on_click(on_button_clicked)
box = widgets.VBox([usernamewidget,passwordwidget,uploadButton])
box

